### Estimativa de Prognóstico de Geração

Desenvolvam um modelo preditivo capaz de estimar a geração futura
de uma usina com potência xxx e localização yyy, considerando a
geração histórica de usinas vizinhas. Esse modelo poderá auxiliar em
planejamentos de expansão e manutenção preventiva das usinas.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df_unidade_consumidora = pd.read_csv('./datasets/unidade_consumidora')
df_geracao = pd.read_csv('./datasets/geracao', chunksize=1000000)
df_usina = pd.read_csv('./datasets/usina')
df_cidade = pd.read_csv('./datasets/cidade')
df_endereco = pd.read_csv('./datasets/endereco')
df_estado = pd.read_csv('./datasets/estado')
df_id_ext = pd.read_csv('./datasets/id_ext')
df_usina_historico = pd.read_csv('./datasets/usina_historico')

In [31]:
# Identificar os valores ausentes
print("Valores ausentes por coluna:")
print(df_usina.isnull().sum())

Valores ausentes por coluna:
id                        0
potencia                  0
unidade_consumidora_id    0
last_plant_history_id     0
dtype: int64


In [23]:
# Preencher valores ausentes de 'potencia' com a mediana
df_usina['potencia'].fillna(df_usina['potencia'].median(), inplace=True)

/tmp/ipykernel_20749/773752065.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_usina['potencia'].fillna(df_usina['potencia'].median(), inplace=True)


In [27]:
# Filtrar as linhas onde 'last_plant_history_id' é NaN
df_usina[df_usina['last_plant_history_id'].isnull()].head()

,id,potencia,unidade_consumidora_id,last_plant_history_id
160039,18935,64.17,21660,NaN
143087,18937,11.37,21661,NaN
78996,18944,27.54,21655,NaN
68619,19905,0.00,22696,NaN
106755,21551,6.00,24624,NaN


In [21]:
df_usina.sort_values(by='id', inplace=True)
df_usina.head()

,id,potencia,unidade_consumidora_id,last_plant_history_id
888316,134,82.50,224,6.296957e+09
853331,143,18.00,237,6.308801e+09
888314,144,70.04,238,6.296957e+09
679634,146,9.52,240,6.308801e+09
490320,147,4.76,241,6.296957e+09


In [ ]:
df_usina.dropna(inplace=True)

,id,potencia,unidade_consumidora_id,last_plant_history_id
888316,134,82.50,224,6.296957e+09
853331,143,18.00,237,6.308801e+09
888314,144,70.04,238,6.296957e+09
679634,146,9.52,240,6.308801e+09
490320,147,4.76,241,6.296957e+09
...,...,...,...,...
59730,1168925,2.36,1346406,6.308532e+09
59719,1168926,7.14,1346407,6.308532e+09
277633,1168927,0.00,1346408,6.308532e+09
282958,1168928,0.00,1346409,6.308500e+09


In [ ]:
# Desafio 2
desafio_2_df = df[['potencia', 'quantidade', 'id_cidade']]
desafio_2_df.groupby(['id_cidade']).mean()

from sklearn.model_selection import train_test_split as ms
from sklearn.linear_model import LinearRegression as lm

# Filtrando outliers
desafio_2_df = desafio_2_df[desafio_2_df['id_cidade'] < 6000]
desafio_2_df = desafio_2_df[desafio_2_df['potencia'] < 200]

# Definindo X e y
X = desafio_2_df[["id_cidade", 'potencia']]
y = desafio_2_df["quantidade"]

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = ms(X, y, test_size=0.4, random_state=42)

# Criando o modelo de regressão linear
regressor = lm()
regressor.fit(X_train, y_train)

# Previsões com o conjunto de teste
y_pred = regressor.predict(X_test)

# Visualizando a regressão no gráfico (Ajuste para múltiplas variáveis)
# Como temos duas variáveis independentes, vamos visualizar a relação para o treino (X_train)
# Vamos gerar previsões para o conjunto de treino com base no modelo ajustado

# Plotando os dados de treino
fig = plt.figure(figsize=(10, 6))

# Visualizando a relação entre 'potencia' e 'quantidade'
plt.scatter(X_train['potencia'], y_train, color='red', label='Dados de Treinamento - Potência')

# Predição para a 'potencia'
plt.plot(X_train['potencia'], regressor.predict(X_train), color='blue', label='Linha de Regressão (Potência)')

# Visualizando a relação entre 'id_cidade' e 'quantidade'
plt.scatter(X_train['id_cidade'], y_train, color='green', label='Dados de Treinamento - Cidade')

# Predição para o 'id_cidade'
plt.plot(X_train['id_cidade'], regressor.predict(X_train), color='orange', label='Linha de Regressão (Cidade)')

# Título e rótulos
plt.title('Gráfico de Regressão Linear')
plt.xlabel('Variáveis Independentes')
plt.ylabel('Quantidade de Energia')
plt.legend()
plt.show()

# Exibindo a equação da reta
a = regressor.coef_  # Coeficientes das variáveis
b = regressor.intercept_  # Intercepto

# Como temos duas variáveis independentes, a equação é:
print(f"A equação da reta é: y = {a[0]:.2f} * id_cidade + {a[1]:.2f} * potencia + {b:.2f}")